In [1]:
import numpy as np
# from scipy.misc import imshow, imsave, imread, imresize
from scipy.ndimage import filters, morphology
# from skimage import filter
from skimage.draw import line
from scipy import ndimage, spatial
import cv2

In [2]:
def houghLines(img):
    w,h = img.shape
    acc=[]
    for i in range(h):
        rr,cc = line(0, i, w-1, h-i-1)
        acc.append(np.sum(img[rr, cc]))
    mi = np.argmax(acc)
    ret = np.zeros(img.shape, dtype=np.bool)
    rr,cc = line(0, mi, w-1, h-mi-1)
    ret[rr,cc]=True
    return ret

In [45]:
def removeLines(img):
    imggray = np.mean(img, -1)
    imfft = np.fft.fft2(imggray)
    imffts = np.fft.fftshift(imfft)

    mags = np.abs(imffts)
    angles = np.angle(imffts)

    visual = np.log(mags)


    visual3 = np.abs(visual.astype(np.int16) - np.mean(visual))

    ret = houghLines(visual3)
    ret = morphology.binary_dilation(ret )
    ret = morphology.binary_dilation(ret )
    ret = morphology.binary_dilation(ret )
    ret = morphology.binary_dilation(ret )
    ret = morphology.binary_dilation(ret )
    ret = morphology.binary_dilation(ret )
#     ret = morphology.binary_dilation(ret )
#     ret = morphology.binary_dilation(ret )
#     ret = morphology.binary_dilation(ret )
#     ret = morphology.binary_dilation(ret )
#     ret = morphology.binary_dilation(ret )
    w,h=ret.shape
    ret[w//2-3:w//2+3, h//2-3:h//2+3]=False


    delta = np.mean(visual[ret]) - np.mean(visual)


    visual_blured = ndimage.gaussian_filter(visual, sigma=4)



    visual[ret] =visual_blured[ret]


    newmagsshift = np.exp(visual)

    newffts = newmagsshift * np.exp(1j*angles)

    newfft = np.fft.ifftshift(newffts)

    imrev = np.fft.ifft2(newfft)

    newim2 =  np.abs(imrev).astype(np.uint8)


    newim2 = np.maximum(newim2, imggray)

    return newim2

In [39]:
image = cv2.imread('/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/Photo 3130 .jpg')[:,:,:3]

In [29]:
img = cv2.imread("/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/Photo 3130 .jpg")[:,:,:3]
imggray = np.mean(image, -1)
imfft = np.fft.fft2(imggray)
mags = np.abs(np.fft.fftshift(imfft))
angles = np.angle(np.fft.fftshift(imfft))
visual = np.log(mags)
visual2 = (visual - visual.min()) / (visual.max() - visual.min())*255
mask = cv2.imread("/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/Photo 3130 .jpg")[:,:,:3]
mask = (np.mean(mask,-1) > 20)
visual[mask] = np.mean(visual)
newmagsshift = np.exp(visual)
newffts = newmagsshift * np.exp(1j*angles)
newfft = np.fft.ifftshift(newffts)
imrev = np.fft.ifft2(newfft)
newim2 = 255 - np.abs(imrev).astype(np.uint8)

In [21]:
# houghLines(newim2)

In [22]:
image.shape

(1080, 810, 3)

In [23]:
newim2.shape

(1080, 810)

In [33]:
# cv2.imwrite("/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/transformedData/Photo 3130 (removedLineFun).jpg",  removedLineFun)

True

In [46]:
removedLineFun = removeLines(image)

In [37]:
# cv2.imwrite("/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/transformedData/Photo 3130 (removedLineFun)2.jpg",  removedLineFun)

True

In [47]:
cv2.imwrite("/home/cwhyse/BloomProj/scribble-stadium-ds/data_management/autopreprocess_testing/data/transformedData/Photo 3130 (removedLineFun)3.jpg",  removedLineFun)

True